In [210]:
import numpy as np
import matplotlib.pyplot as plt

In [211]:
def convolve(sheet, kernel, stride = 1, padding = 2):
    kernel_size = kernel.shape[0]
    output_image = np.zeros((int((sheet.shape[0] - kernel_size + 2*padding)/stride) + 1, int((sheet.shape[1] - kernel_size + 2*padding)/stride) + 1))
    sheet = np.pad(sheet, padding, 'constant')
    for i in range(output_image.shape[0]):
        for j in range(output_image.shape[1]):
            output_image[i, j] = np.sum(sheet[i*stride:i*stride+kernel_size, j*stride:j*stride+kernel_size] * kernel)
            # print(output_image[i, j])
    return output_image

In [212]:
kernel = np.ones((3, 3))
op = convolve(np.ones((5, 5)), kernel, stride = 2, padding = 0)
print(np.ones((5, 5)))
print(kernel)
print(op.shape)
print(op)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
(2, 2)
[[9. 9.]
 [9. 9.]]


In [213]:
def convolve_image(image, kernel, stride, padding):
    channels = image.shape[2]
    if(kernel.shape[2] != channels):
        print("Error: Kernel and Image not conformable")
        return
    else:
        output_size = int((image.shape[0] - kernel.shape[0] + 2*padding)/stride) + 1
        output_image = np.zeros((output_size, output_size))
        channel_sum = np.zeros_like(output_image)
        for i in range(channels):
            channel_sum += convolve(image[:, :, i], kernel[:, :, i], stride, padding)
        output_image = channel_sum
        return output_image

In [214]:
convolve_image(np.ones((5, 5, 3)), np.ones((3, 3, 3)), 1, 0)

array([[27., 27., 27.],
       [27., 27., 27.],
       [27., 27., 27.]])

In [215]:
def conv2d(input_volume, out_channels, kernel_size, stride = 1, padding = 0):
    in_shape= input_volume.shape[0]
    in_channels = input_volume.shape[2]
    kernels = np.ones((out_channels, kernel_size, kernel_size, in_channels))
    output_size = int((in_shape - kernel_size + 2*padding)/stride) + 1
    output_volume = np.zeros((output_size, output_size, out_channels))
    print(output_volume.shape)
    for i in range(out_channels):
        output_volume[:,:,i] = convolve_image(input_volume, kernels[i], stride, padding)
    return output_volume

In [216]:
conv2d(np.ones((5, 5, 3)), 7, 3, 1, 0)

(3, 3, 7)


array([[[27., 27., 27., 27., 27., 27., 27.],
        [27., 27., 27., 27., 27., 27., 27.],
        [27., 27., 27., 27., 27., 27., 27.]],

       [[27., 27., 27., 27., 27., 27., 27.],
        [27., 27., 27., 27., 27., 27., 27.],
        [27., 27., 27., 27., 27., 27., 27.]],

       [[27., 27., 27., 27., 27., 27., 27.],
        [27., 27., 27., 27., 27., 27., 27.],
        [27., 27., 27., 27., 27., 27., 27.]]])

In [217]:
def pool_image(input_volume, kernel_size, stride, padding, mode = "avg"):
    channels = input_volume.shape[2]
    output_size = int((input_volume.shape[0] - kernel_size + 2*padding)/stride) + 1
    output_volume = np.zeros((output_size, output_size, channels))
    for i in range(channels):
        if(mode == "avg"):
            output_volume[:, :, i] = convolve(input_volume[:, :, i], np.ones((kernel_size, kernel_size)), stride, padding)/kernel_size**2
        elif(mode == "max"):
            for j in range(output_size):
                for k in range(output_size):
                    output_volume[j, k, i] = np.max(input_volume[j*stride:j*stride+kernel_size, k*stride:k*stride+kernel_size, i])
    return output_volume

In [218]:
x = np.random.randn(5, 5, 3)
display(x.reshape(3, 5,5))
pool_image(x, 3, 1, 0, "max").reshape(3, 3, 3)

array([[[-0.14715925,  0.16488345,  0.39500685, -0.13417148,
         -0.77588273],
        [ 0.63145276, -2.69188579,  0.14151972, -1.32342799,
          1.23965853],
        [ 1.0385029 , -0.44787742,  0.19798162,  0.12987171,
          0.74462425],
        [-0.83144586, -0.66190902, -0.28706348,  1.79288497,
         -1.13320175],
        [ 0.71621205,  1.14986703,  0.0679152 ,  2.02485132,
          0.07835828]],

       [[ 1.44606737, -0.33183818, -0.0410086 , -1.07074213,
          1.33963081],
        [ 0.15203912,  0.33572795, -0.90454091, -1.10790634,
         -0.66144087],
        [-1.53223705,  0.36237228,  0.16530729,  0.74297814,
         -0.60681426],
        [ 1.96452149,  1.2372718 ,  0.7471    ,  0.28996784,
          0.06943595],
        [-0.23813912,  0.95289716,  0.73390451,  1.52819756,
          0.02593017]],

       [[ 0.10378245, -1.4018487 ,  1.3421518 ,  0.47365956,
          1.52033442],
        [-0.14308553, -0.18800931, -1.35376598, -2.09005835,
         -0

array([[[1.79288497, 0.33572795, 2.02485132],
        [1.79288497, 1.96452149, 2.02485132],
        [1.23965853, 1.96452149, 2.02485132]],

       [[1.79288497, 1.3421518 , 2.02485132],
        [1.79288497, 1.96452149, 2.02485132],
        [1.52033442, 1.96452149, 2.02485132]],

       [[1.52819756, 1.4563728 , 0.74297814],
        [1.52819756, 1.96452149, 1.2372718 ],
        [1.52033442, 1.96452149, 1.2372718 ]]])